In [1]:
import matplotlib.pyplot as plt
from glob import glob

import cv2
import os
import numpy as np
import time
import base64
import codecs, json
import pickle

from PIL import Image

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
#from keras_retinanet.utils.gpu import setup_gpu
from keras.models import Model



# object detection
os.path.join('./keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

model_path = os.path.join('./keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')


Using TensorFlow backend.


In [2]:
dataset_path = "./original_train/"
output_path = "./detected_data/detected_from_train/"

In [3]:

retina_model = models.load_model(model_path, backbone_name='resnet50')
os.chdir(dataset_path)
dataset_list = os.listdir(os.getcwd())
os.chdir('../')


from object_detection2 import object_detection
detected_train_formats = object_detection(retina_model, dataset_list, dataset_path, output_path)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
handling person000031.png
image shape: (428, 640, 3)
resized image size: (800, 1196, 3) scale: 1.8691588785046729

/home/plass-heesu/anaconda3/envs/tf1.15/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
(1, 300, 4) (1, 300) (1, 300)
processing time:  3.229447603225708
handling

In [4]:
import pickle
with open("detected_train_formats.p", "wb") as out:
    pickle.dump(detected_train_formats, out)

In [6]:

with open("./detected_train_formats.p","rb") as f:
    detected_train_formats = pickle.load(f)

from glob import glob
image_path = [format["objectImagePath"] for format in detected_train_formats]

In [7]:
detected_images_train = [Image.open(path) for path in image_path]
len(detected_images_train)

5423

In [8]:
image_path[0]

'./detected_data/detected_from_train/clock1_path: (.-original_train-person000031.png).jpg'

In [9]:
# with open("detected_train.p", "rb") as inFile:
#     detected_train_formats = pickle.load(inFile)

from ImageRetrievalClass import ImageRetrievalClass

retrieval = ImageRetrievalClass("ResNet50v2", True, False)
retrieval.readTrainSet(detected_images_train)


Reading train images
train image shape = (512, 512, 3)


In [10]:
retrieval.buildModel()


Loading ResNet50v2 pre-trained model...
Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 258, 258, 64) 0           conv1_conv[0][0]                 
_________________________________________________

In [11]:
X_train = retrieval.trainTransform()

Applying image transformer to training images...
 -> X_train.shape = (5423, 512, 512, 3)


# X_train(feature map 추출전 transform된 객체들) 저장

In [12]:
np.save("X_train.npy", X_train)

In [13]:
X_train = np.load("X_train.npy")

In [14]:
type(X_train)

numpy.ndarray

In [15]:
X_train.shape

(5423, 512, 512, 3)

# E_train (추출한 feature map) 생성 및 저장

In [16]:
E_train = retrieval.predictTrain(X_train)

Inferencing embeddings using pre-trained model...
 -> E_train.shape = (5423, 16, 16, 2048)
 -> E_train_flatten.shape = (5423, 524288)


In [17]:
# np.save("E_train.npy", E_train)

In [18]:
type(E_train)

numpy.ndarray

In [19]:
E_train.shape

(5423, 16, 16, 2048)

In [20]:
E_train_flatten = retrieval.E_train.reshape((-1, np.prod(retrieval.output_shape_model)))
E_train_flatten.shape

(5423, 524288)

In [21]:
# np.save("E_train_flatten.npy", E_train_flatten)

# 객체 각각의 npy 파일 저장

In [22]:

# for format in detected_train_formats:
#     print(format["objectPath"][36:-4])

In [26]:
for idx, format in enumerate(detected_train_formats):
    filename = format["objectImagePath"][36:-4]
    npyFilename = "./npyFiles/"+filename+".npy"
    np.save(npyFilename, E_train[idx])

# JSON 파일 저장 

In [27]:
import json
for idx, format in enumerate(detected_train_formats):
    npyFilename = "./npyFiles/"+format["objectImagePath"][36:-4]+".npy"
    jsonDict = {
        "objectID":format["objectID"],
        "location":format["location"],
        "tag":format["tag"],
        "objectImagePath" : format["objectImagePath"],
        "npyPath": npyFilename,
        "IMG_URL" : format["IMG_URL"]
    }

    filename = "./jsonFiles/"+format["objectImagePath"][36:-4]+".json"

    with open(filename, "w") as outFile:
        json.dump(jsonDict, outFile, indent=2)